In [144]:
import pandas as pd
import numpy as np
import folium
import hdbscan
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import MultiPoint
from matplotlib import cm, colors

In [145]:
# List both folders you want to read from
folder_paths = [
    "Meetpunten/Waterschap Brabantse Delta",
    "Meetpunten/Waterschap de Dommel",
]

all_csv_files = []
for folder_path in folder_paths:
    # Collect CSV files from each folder
    csv_files = [
        os.path.join(folder_path, f) 
        for f in os.listdir(folder_path) 
        if f.endswith(".csv")
    ]
    all_csv_files.extend(csv_files)

# Now read them all into a list of DataFrames
dfs = []
for file_path in all_csv_files:
    df_temp = pd.read_csv(file_path, sep=";", encoding="utf-8")
    dfs.append(df_temp)

# Finally, combine them
meetpunten_all = pd.concat(dfs, ignore_index=True)


In [146]:
# Extract RD coordinates
X = meetpunten_all[["GeometriePuntX_RD", "GeometriePuntY_RD"]].values

# Fit HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean')
cluster_labels = clusterer.fit_predict(X)

# Add result to DataFrame
meetpunten_all["hdbscan_cluster"] = cluster_labels


In [147]:
transformer = Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)

# Colormap
unique_hdb = sorted(meetpunten_all["hdbscan_cluster"].unique())
hdb_color_map = {
    cid: colors.rgb2hex(cm.tab20(i % 20)[:3]) for i, cid in enumerate(unique_hdb) if cid != -1
}
hdb_color_map[-1] = "#999999"  # noise

# Build map
m = folium.Map(location=[51.6, 5.0], zoom_start=9)

for _, row in meetpunten_all.iterrows():
    lon, lat = transformer.transform(row["GeometriePuntX_RD"], row["GeometriePuntY_RD"])
    cid = row["hdbscan_cluster"]
    marker_color = hdb_color_map[cid]
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=4,
        color=marker_color,
        fill=True,
        fill_opacity=0.85,
        popup=f"Cluster {cid}" if cid != -1 else "Noise",
        tooltip=f"Cluster {cid}" if cid != -1 else "Noise"  # <-- Tooltip for hover
    ).add_to(m)

m.save("hdbscan_meetpunten_map.html")
print("Saved map as 'hdbscan_meetpunten_map.html'")

Saved map as 'hdbscan_meetpunten_map.html'


In [148]:
# 4. Get the full set of years
all_years = set(meetpunten_all["Meetjaar"].unique())

# 5. For each cluster, gather the distinct years
cluster_years = (
    meetpunten_all.groupby("hdbscan_cluster")["Meetjaar"]
    .apply(set)
    .reset_index(name="YearsPresent")
)

# 6. Check which clusters cover *all* years
cluster_years["HasAllYears"] = cluster_years["YearsPresent"].apply(
    lambda yrs: all_years.issubset(yrs)
)

# 7. Filter to clusters with all years present
clusters_with_all_years = cluster_years[cluster_years["HasAllYears"]]
clusters_list = clusters_with_all_years["hdbscan_cluster"].unique()

print("Clusters that have at least one measurement point in EVERY year:")
print(clusters_with_all_years)

Clusters that have at least one measurement point in EVERY year:
     hdbscan_cluster                                       YearsPresent  \
0                 -1  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
30                29  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
39                38  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
45                44  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
75                74  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
76                75  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
88                87  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
90                89  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
95                94  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
138              137  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
149              148  {2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...   
152              151  {2016, 2017, 

In [149]:
# 6. Filter original data to include only those clusters
meetpunten_filtered = meetpunten_all[meetpunten_all["hdbscan_cluster"].isin(clusters_list)]

# 7. Build the map
m = folium.Map(location=[51.6, 5.0], zoom_start=9)

# Create a colormap for the filtered clusters
unique_hdb = sorted(meetpunten_filtered["hdbscan_cluster"].unique())
# You can pick any palette you prefer; here we use tab20
hdb_color_map = {
    cid: colors.rgb2hex(cm.tab20(i % 20)[:3]) for i, cid in enumerate(unique_hdb) if cid != -1
}
hdb_color_map[-1] = "#999999"  # Noise cluster color

transformer = Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)

for _, row in meetpunten_filtered.iterrows():
    x_rd = row["GeometriePuntX_RD"]
    y_rd = row["GeometriePuntY_RD"]
    
    # Convert RD (EPSG:28992) to WGS84 (EPSG:4326)
    lon, lat = transformer.transform(x_rd, y_rd)
    
    cid = row["hdbscan_cluster"]
    marker_color = hdb_color_map.get(cid, "#999999")

    folium.CircleMarker(
        location=[lat, lon],
        radius=4,
        color=marker_color,
        fill=True,
        fill_opacity=0.9,
        popup=f"Cluster: {cid} | Year: {row['Meetjaar']}",
        tooltip=f"Cluster {cid}"
    ).add_to(m)

m.save("clusters_with_all_years_map.html")
print("Saved map as 'clusters_with_all_years_map.html'")

Saved map as 'clusters_with_all_years_map.html'


In [150]:
food_df = pd.read_csv("food_forests.csv", sep=";", encoding="latin-1")
# Drop the last row:
food_df = food_df.iloc[:-1]

food_df.head(15)

,ï»¿Name,Address,Latitude,Longitude
0,Food forest Vierhoeven Roosendaal,Nispenseweg 1A,51.513.182.820.318.000,4.474.986.454.643.190
1,Food Forest Wilderness,De Ruttestraat 11,51.420.605.340.640.300,5.192.964.546.958.110
2,Food forest Haverleij,P793+8M,51.718.596,5.252.470
3,Food forest Kleindongen,Klein Dongenseweg 81,51.641.617,4.952.325
4,Dream forest Leende,NaN,51.359.917.409.725.700,5.569.345.605.993.530
5,'t Mortelke,Het Mortelke 3,51.295.764,5.423.690
6,Voedselbos het Avontuur,Gaardsebaan 21,51.453.549.227.638.400,46.839.589.554.104.700
7,Voedselbos Biezenmortel,Biezenmortelsestraat 47,51.632.853,5.179.436
8,Voedselbos âs Heeren Vruchten,NaN,51.533.894.791.614.000,4.670.422.404.581.660
9,Voedselbos Woest,Heintje Davidsstraat 7,51.734.895.499.018.900,5.343.549.148.539.780


In [151]:
def fix_coordinate(coord_str):
    """
    Turn something like '51.513.182.820.318.000' into float(51.513182820318).
    Also handle rows that might be '51.718.596' (-> 51.718596)
    or might already be '51.395529' (already a proper float).
    """
    # If it already parses as a float (only one dot or no dot), we can just return that.
    try:
        return float(coord_str)  # If this works without error, we're done
    except ValueError:
        pass

    # If we get here, it means there are multiple dots, so let's merge them
    parts = coord_str.split('.')
    if len(parts) > 1:
        # For example: '51', '513', '182', '820', '318', '000'
        first_part = parts[0]           # '51'
        second_part = parts[1]         # '513'
        remainder = ''.join(parts[2:]) # '182820318000'
        cleaned_str = f"{first_part}.{second_part}{remainder}"
        return float(cleaned_str)
    else:
        # Worst case fallback
        return float(coord_str or 0)

food_df["Latitude_clean"] = food_df["Latitude"].apply(fix_coordinate)
food_df["Longitude_clean"] = food_df["Longitude"].apply(fix_coordinate)


In [152]:
print(food_df[["ï»¿Name", "Latitude", "Latitude_clean", "Longitude", "Longitude_clean"]].head(15))

                              ï»¿Name                Latitude  Latitude_clean  \
0   Food forest Vierhoeven Roosendaal  51.513.182.820.318.000       51.513183   
1              Food Forest Wilderness  51.420.605.340.640.300       51.420605   
2               Food forest Haverleij              51.718.596       51.718596   
3             Food forest Kleindongen              51.641.617       51.641617   
4                 Dream forest Leende  51.359.917.409.725.700       51.359917   
5                         't Mortelke              51.295.764       51.295764   
6             Voedselbos het Avontuur  51.453.549.227.638.400       51.453549   
7             Voedselbos Biezenmortel              51.632.853       51.632853   
8     Voedselbos âs Heeren Vruchten  51.533.894.791.614.000       51.533895   
9                    Voedselbos Woest  51.734.895.499.018.900       51.734895   
10           The Eibernest Foundation              51.395.529       51.395529   

                 Longitude 

In [153]:
# Suppose `m` is already created, e.g.:
# m = folium.Map(location=[51.6, 5.0], zoom_start=9)

for _, row in food_df.iterrows():
    # Use the cleaned lat/long columns
    lat = row["Latitude_clean"]
    lon = row["Longitude_clean"]
    name = row["ï»¿Name"]  # or however you store the forest name
    
    # Add a marker (or CircleMarker, if you prefer)
    folium.Marker(
        location=[lat, lon],
        popup=name,          # text shown on click
        tooltip=name         # text on hover
    ).add_to(m)

# Finally, save or display the map
m.save("clusters_with_foodforests_map.html")
print("Map with Food Forests saved as 'clusters_with_foodforests_map.html'")


Map with Food Forests saved as 'clusters_with_foodforests_map.html'
